## Imports and Reading Datasets

In [7]:
!pip install --user uszipcode
from uszipcode import SearchEngine
import pandas as pd
import numpy as np
import random

search = SearchEngine(simple_zipcode=True)
df = pd.read_csv('/datasets/iowaliquor.csv')[['Date', 'Store Name', 'Zip Code', 'Sale (Dollars)', 'County']]
election_df = pd.read_csv('usa-2016-presidential-election-by-county.csv', sep=';')
election_df = election_df[election_df['State'] == 'Iowa']

## Fill Missing Zip Codes

In [2]:
unqs = df[df["Zip Code"].isna()]["Store Name"].unique() # get unique store names
cities = [store.split("/")[-1].strip().lower() for store in unqs if "/" in store] # extract city name
if "mlk" in cities:
    cities.remove("mlk")
subset = df[df["Store Name"].str.contains("/") & (df["Zip Code"].isna())] 
city_dict = {}
for city in set(cities): # get zip code from rows with cities in store name
    city_dict[city] = search.by_city_and_state(city, "iowa")[0].zipcode
city_dict["mlk"] = '50310'
df.loc[subset.index, 'Zip Code'] = subset["Store Name"].str.split("/").str[-1].str.strip().str.lower().map(city_dict) 

## Aggregate and Get Zip Code Statistics

In [3]:
df["YearMonth"] = df["Date"].str.split("/").str[2] + "-" + df["Date"].str.split("/").str[0]

df = df[df["Zip Code"].notna()]
df["Zip Code"] = df["Zip Code"].replace("712-2", 51529)
df["Zip Code"] = df["Zip Code"].astype(int)

countyDict = {}
popDict = {}

for zc in df["Zip Code"].unique():
    stats = search.by_zipcode(zc)
    if stats.state == "IA":
        countyDict[zc] = stats.county
        popDict[zc] = stats.population

In [38]:
totalZip = pd.DataFrame(df.groupby(["YearMonth", "Zip Code"])["Sale (Dollars)"].sum())
totalZip.reset_index(inplace=True)
totalZip["Population"] = totalZip["Zip Code"].map(popDict)

## County Aggregate & Add Election Data

In [61]:
totalZip["County"] = totalZip["Zip Code"].map(countyDict)
totalZip.dropna(inplace=True)
totalCounty = pd.DataFrame(totalZip.groupby(["YearMonth", "County"])["Sale (Dollars)"].sum())
totalCounty.reset_index(inplace=True)
totalCounty["Month"] = totalCounty["YearMonth"].str.split("-").str[-1]

election_df["County"] = election_df["County"].str.replace(', Iowa', "")
totalCounty = totalCounty.merge(election_df, on="County")
totalCounty["SalesPerCapita"] = totalCounty["Sale (Dollars)"] / totalCounty["Total Population"]

## Transform Variables

In [161]:
totalCounty = pd.read_csv("totalCounty.csv").drop("Unnamed: 0", axis=1)
totalCounty["LogSPC"] = np.log(totalCounty["SalesPerCapita"])
totalCounty["LogBachelors"] = np.log(totalCounty["At Least Bachelors's Degree"])
totalCounty["LogPop"] = np.log(totalCounty["Total Population"])
totalCounty["YearsPast2012"] = totalCounty["YearMonth"].str.split("-").str[0].astype(int) - 2012
totalCounty["Adult.smoking"] = totalCounty["Adult.smoking"].replace(np.nan, totalCounty["Adult.smoking"].mean())
totalCounty["Intercept"] = 1

## Linear Regression

In [208]:
cols = ["Intercept", "Month", "Median Age", "Adult.obesity", "Diabetes", "White", "Less Than High School Diploma", "LogBachelors", "LogPop", "Median Earnings 2010", "YearsPast2012", "Adult.smoking", "Republicans 2012"]
X = totalCounty[cols]
X = np.array(pd.get_dummies(X))
y = np.array(totalCounty['LogSPC'])

b = np.linalg.inv(X.T@X)@X.T@y

np.sqrt(((np.e**y - np.e**(X@b))**2).mean())

2.7192606979124596

## Cross Validation

In [181]:
indices = [i for i in range(X.shape[0])]
group_size = X.shape[0] // 4
groups = []

for i in range(3):
    cur_group = random.sample(indices, group_size)
    groups.append(cur_group)
    for ind in cur_group:
        indices.remove(ind)
        
groups.append(indices)

In [196]:
scores = []
for i in range(4):
    train_inds = sum([groups[j] for j in range(len(groups)) if j != i], [])
    X_train = X[train_inds]
    X_test = X[groups[i]]
    y_train = y[train_inds]
    y_test = y[groups[i]]
    
    b = np.linalg.inv(X_train.T@X_train)@X_train.T@y_train
    scores.append(np.sqrt(((np.e**y_test - np.e**(X_test@b))**2).mean()))
    
np.mean(scores)

2.716446187274086

## AIC Score

In [209]:
indices = [i for i in range(X.shape[0])]
group_size = X.shape[0] // 4
groups = []

for i in range(3):
    cur_group = random.sample(indices, group_size)
    groups.append(cur_group)
    for ind in cur_group:
        indices.remove(ind)
        
groups.append(indices)

aic_scores = []
for i in range(4):
    train_inds = sum([groups[j] for j in range(len(groups)) if j != i], [])
    X_train = X[train_inds]
    X_test = X[groups[i]]
    y_train = y[train_inds]
    y_test = y[groups[i]]
    
    b = np.linalg.inv(X_train.T@X_train)@X_train.T@y_train
    residuals = np.e**y_test - np.e**(X_test@b)
    aic_scores.append(np.std(residuals)*(residuals**2).sum() + 2*X_train.shape[1])
    
np.mean(aic_scores)

51425.53576924244